In [ ]:
# Necassary library imports
import os
import time
import logging
import json
from tqdm import tqdm
from datetime import datetime

# Importing Modular File Functions
from api.test import test_api_connection
from task_generator.task_manager import repair_example
from config.model import BASE_DIR

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# Load task list from JSON file in current directory
TASK_FILE = BASE_DIR / "erraneous_examples_set3.json"  # or "failed_examples.json"

"""
    Load all failed examples from the provided JSON file.
    Returns a list of dicts, each representing a failed example.
"""
if TASK_FILE.exists():
    with open(TASK_FILE, "r", encoding="utf-8") as f:
        task_data = json.load(f)
        TOTAL_EXAMPLES = len(task_data)
        logger.info(f"📋 Loaded {TOTAL_EXAMPLES} examples from {TASK_FILE.name}")
else:
    TOTAL_EXAMPLES = 0
    print(f"⚠️ Task file not found at {TASK_FILE.name}, TOTAL_EXAMPLES set to 0")

In [ ]:
def repair_examples():
    """Main function to orchestrate the examples repair process."""
    logger.info(f"Starting the repair process of {TOTAL_EXAMPLES} Raspberry Pi code examples")
    
    # Current user info for metadata
    current_user = os.environ.get("USER", "Unknown")
    current_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    
    logger.info(f"Current Date and Time (UTC): {current_datetime}")
    logger.info(f"Current User's Login: {current_user}")
    
    # First test the API connection
    if not test_api_connection():
        logger.error(f"API Connection failed")
        return
    
    # Setup progress tracking
    successful = 0
    failed = 0
    
    # Initialize progress bar
    pbar = tqdm(TOTAL_EXAMPLES, desc="Repairing examples")
    
    # Process tasks
    start_time = time.time()
    for i, example in enumerate(task_data):
        logger.info(f"Repairing example {i+1}/{TOTAL_EXAMPLES}.")
        try:
            if repair_example(example):
                successful += 1
            else:
                failed += 1
        except Exception as e:
            logger.error(f"❌ Exception occurred while repairing ID {i}: {str(e)}")
            failed += 1
        
        # Update progress bar
        pbar.update(1)
        pbar.set_description(f"Generated: {successful}, Failed: {failed}")
        
        # Calculate and display ETA
        elapsed = time.time() - start_time
        avg_time_per_item = elapsed / (i + 1)
        remaining_items = TOTAL_EXAMPLES - (i + 1)
        eta_seconds = avg_time_per_item * remaining_items
        
        if i % 10 == 0 and i > 0:
            eta_hours = int(eta_seconds // 3600)
            eta_minutes = int((eta_seconds % 3600) // 60)
            logger.info(f"Progress: {i+1}/{TOTAL_EXAMPLES} examples. Estimated time remaining: {eta_hours}h {eta_minutes}m")
            
    pbar.close()
    
    # Final report
    end_time = time.time()
    total_time = end_time - start_time
    hours = int(total_time // 3600)
    minutes = int((total_time % 3600) // 60)
    seconds = int(total_time % 60)
    
    logger.info(f"Generation complete: {successful} successful, {failed} failed")
    logger.info(f"Total execution time: {hours}h {minutes}m {seconds}s")
    logger.info(f"Examples saved in {os.path.abspath(BASE_DIR)}")  

In [ ]:
if __name__ == "__main__":
    repair_examples()